In [1]:
import boto3
import json

client = boto3.client("sagemaker-runtime")
endpoint13b_name = 'jumpstart-dft-llama-codellama-13b-i-20240210-203037'

In [2]:
def query_endpoint(payload):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(
        EndpointName=endpoint13b_name,
        ContentType='application/json',
        Body=json.dumps(payload).encode('utf-8'),
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

In [3]:
def print_completion(prompt: str, response: str) -> None:
    bold, unbold = '\033[1m', '\033[0m'
    print(f"{bold}> Input{unbold}\n{prompt}{bold}\n> Output{unbold}\n{response[0]['generated_text']}\n")

In [5]:
from typing import List, Dict

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions for Code Llama.
    
    The model only supports 'system', 'user' and 'assistant' roles, starting with 'system', then 'user' and 
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt: List[str] = []

    if instructions[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", instructions[0]["content"], "\n<</SYS>>\n\n", instructions[1]["content"]])
        instructions = [{"role": instructions[1]["role"], "content": content}] + instructions[2:]

    for user, answer in zip(instructions[::2], instructions[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

    prompt.extend(["<s>", "[INST] ", (instructions[-1]["content"]).strip(), " [/INST] "])

    return "".join(prompt)

In [11]:
question = "Write a python function for bubble sort algorithm"

In [12]:
messages = [
      {
            "role": "system",
            "content": "You are an expert programmer that helps to write Python code based on the user request, with concise explanations. Don't be too verbose.",
      },
      {
            "role": "user",
            "content": question ,
      }
]

prompt = format_instructions(messages)

In [13]:
prompt

"<s>[INST] <<SYS>>\nYou are an expert programmer that helps to write Python code based on the user request, with concise explanations. Don't be too verbose.\n<</SYS>>\n\nWrite a python function for bubble sort algorithm [/INST] "

In [14]:
payload = {"inputs": prompt, "parameters": {"max_new_tokens": 256, "temperature": 0.2, "top_p": 0.9}}
response = query_endpoint(payload)
print_completion(prompt, response)

> Input
<s>[INST] <<SYS>>
You are an expert programmer that helps to write Python code based on the user request, with concise explanations. Don't be too verbose.
<</SYS>>

Write a python function for bubble sort algorithm [/INST] 
> Output
```
def bubble_sort(arr):
    n = len(arr)
    for i in range(n-1):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr
```
This function takes an array as input and sorts it using the bubble sort algorithm. The algorithm works by iterating through the array and comparing adjacent elements. If an element is greater than the next element, they are swapped. This process is repeated until the array is sorted. The function returns the sorted array.

